This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [2]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'hWqYzzL73U8Cp-y81W_T'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [3]:
# First, import the relevant modules
import requests
import json

## Q1 solution 
* collecting data from Quandl API of stock AFX_X by using request library in python 

In [4]:
payload = {"api_key":'hWqYzzL73U8Cp-y81W_T',
           "dataset_code":"AFX_X",
           "database_code":"EOD",
           "name":"FB",
           "start_date":"2017-01-01",
           "end_date":"2017-12-31"}

r = requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?api_key=hWqYzzL73U8Cp-y81W_T",params=payload)


In [5]:
r.url

'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?api_key=hWqYzzL73U8Cp-y81W_T&api_key=hWqYzzL73U8Cp-y81W_T&dataset_code=AFX_X&database_code=EOD&name=FB&start_date=2017-01-01&end_date=2017-12-31'

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

## Q2 solution 
* converting json object into python Dict 

In [6]:
# converting json string to dict 
data = json.loads(r.text)

In [7]:
type(data)

dict

In [8]:
data.keys()

dict_keys(['dataset_data'])

In [9]:
data['dataset_data'].keys()

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])

In [10]:
data['dataset_data']["column_names"]

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [11]:
from collections import OrderedDict

In [12]:
columns_names = data['dataset_data']["column_names"][1:]
columns_names

['Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [13]:
all_data = OrderedDict()
for current_day_date in data['dataset_data']["data"]:
    #print(current_day_date)
    #print(len(columns_names),len(current_day_date[1:]))
    all_data[current_day_date[0]] = OrderedDict(list(zip(columns_names,current_day_date[1:])))

In [14]:
# now we have data stored day wise and dict is orderd so dates position have not changed
all_data['2017-12-29']

OrderedDict([('Open', 51.76),
             ('High', 51.94),
             ('Low', 51.45),
             ('Close', 51.76),
             ('Change', None),
             ('Traded Volume', 34640.0),
             ('Turnover', 1792304.0),
             ('Last Price of the Day', None),
             ('Daily Traded Units', None),
             ('Daily Turnover', None)])

## Q3 solution 
* highest and lowest open price in 2017

In [15]:
# now i will loop over this 
open_price = []
for trading_day in all_data:
    
    # there are nan values in open prices so making sure values wich are floats only those value to be appended
    open_p = all_data[trading_day]["Open"]
    if isinstance(open_p,float):  
        open_price.append(open_p)

In [16]:
print("highest open price in 2017:",max(open_price))
print("lowest open price in 2017:",min(open_price))

highest open price in 2017: 53.11
lowest open price in 2017: 34.0


## Q4 solution 
* What was the largest change in any one day (based on High and Low price)

In [17]:
max_diff = 0
for trading_day in all_data:
    current_day_high = all_data[trading_day]["High"]
    current_day_low = all_data[trading_day]["Low"]
    
    current_day_diff = current_day_high - current_day_low
    
    if current_day_diff > max_diff:
        max_diff = current_day_diff
    
    

In [18]:
print("biggest change in one day in 2017 is:",max_diff)

biggest change in one day in 2017 is: 2.8100000000000023


## Q5 solution 
* what is the largest change between any two days

In [19]:
max_diff = 0
for count,trading_day in enumerate(all_data):
    
    if count == (len(all_data) - 1):
        break 

    # to calculate diff between two trading days i have take diffrence  of current day close  - previous day close 
    current_day_close = all_data[trading_day]["Close"]
    

    # getting previous day date + 1 because dates our in desecding order 
    previous_day_date = list(all_data.keys())[count + 1]
    

    # getting previous day close price 
    previous_day_close = all_data[previous_day_date]["Close"]
    

    # taking diffrence 
    current_diff = abs(current_day_close - previous_day_close)
    
    
    if current_diff > max_diff:
        max_diff = current_diff
        

    

In [20]:
print("largest change between two days is based on close price:",max_diff)

largest change between two days is based on close price: 2.559999999999995


## Q6 solution 
* average daily trading volume in 2017

In [21]:
volume = []

for trading_day in all_data:
    volume.append(all_data[trading_day]['Traded Volume'])

In [22]:
average_volume  = sum(volume)/len(volume)
print("Average daily trading volume in 2017 is:",average_volume)

Average daily trading volume in 2017 is: 89124.33725490196


## Q7 solution 
* median trading volume in 2017  

In [23]:
def find_median(series):
    series = sorted(series)
                 
    if len(series) % 2 == 0:
        return (series[int((len(series)/2) - 1)] + series[int(len(series)/2)])/2
    else:
        return series[((len(series)//2) + 1) - 1]
        
        

In [24]:
print("median trading volume for 2017:",find_median(volume))

median trading volume for 2017: 76286.0
